<a href="https://colab.research.google.com/github/kimdonggyu2008/audio_deeplearning_python/blob/main/%EC%88%9C%EC%A0%84%ED%8C%8C%2C_%EC%97%AD%EC%A0%84%ED%8C%8C%2C_%ED%95%99%EC%8A%B5(%EC%98%A4%EB%A5%98_%EC%A1%B4%EC%9E%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from random import random

#더 많은 층수를 가지는 mlp?
class MLP(object):
#3개 입력을 받고 2개의 출력을 시행함
  def __init__(self,num_inputs=3,hidden_layers=[3,3], num_outputs=2):
    #입력 갯수, 레이어 갯수, 출력 데이터 갯수 지정

    self.num_inputs=num_inputs
    self.hidden_layers=hidden_layers# 첫번째 층이 3개 뉴런, 두번째 층이 3개 뉴런을 가지게 만듦
    self.num_outputs=num_outputs

    layers = [num_inputs] + hidden_layers + [num_outputs]

    weights=[] #랜덤한 가중치용 배열 생성
    for i in range(len(layers)-1):
      w=np.random.rand(layers[i],layers[i+1])#i번째 레이어 -> i+1번째 레이어로 넘어가는 과정의 가중치 갯수 세기
      weights.append(w)# 다음 층으로 넘어갈때 각 레이어별 갯수가 맞아야 함
    self.weights=weights #현재 층에서의 가중치

#각 레이어 간 기울기 배열 생성(레이어 사이이므로 레이어 갯수 -1이 되어야 함)
    derivatives=[]
    for i in range(len(layers)-1):#층 간 가중치의 도함수를 계산
      d=np.zeros((layers[i],layers[i+1]))#앞 출력값의 갯수, 뒷 입력값의 갯수 크기만큼의 도함수 행렬 생성
      derivatives.append(d)#해당 레이어 -> 다음 레이어로 넘어가는 과정에서, 각 가중치에 대한 그레디언트 행렬
    self.derivatives=derivatives #현재 층->다음 층에서의 그레디언트
      #최종적으로는 각 층간 가중치 배열을 가지게 됨. 2개 레이어면 1개의 가중치 배열만 가지게 됨

#각 레이어의 활성화 값 지정

    activations=[]
    for i in range(len(layers)): #각 층에 대한 활성화 값
      a=np.zeros(layers[i])#해당 층이 가지는 가중치 크기만큼의  zeros배열
      activations.append(a)#추가
    self.activations=activations#현재 층에서의 활성화 값
    #각 층의 최종 활성화 값들을 가지는 배열이 됨


  #순전파 학습
  def forward_propagate(self,inputs):#시그모이드가 적용된 input값 사용
    activations=inputs#활성화 함수
    self.activations[0]=inputs
    for i, w in enumerate(self.weights):# 현재 층에서의 가중치를 가져옴
      net_inputs=np.dot(activations,w)#현재 층의 input값과 현재 층의 가중치 행렬곱 , act 2층 * weight 2층
      activations=self._sigmoid(net_inputs)#주어진 행렬곱의 시그모이드로 구별시킴, 시그모이드(net_input(2층의 결괏값))
      self.activations[i+1]=activations #시그모이드가 적용된 행렬곱을 저장, act 3층 = 2층의 시그모이드(net_input(2층의 결괏값))
      #예시) 3층의 결괏값 = activation 2층 * weight 2층
      # activation 3층 = 시그모이드(3층의 결괏값)

    return activations



#역전파는 f(x) * (1-f(x))의 형태로, 실제값-최종 결과의 지분을 각 가중치에 따라 나눈다고 생각하면 될듯
#해당 결과가 나오기 위해 통한 경로의 오류의 가중치 값들은 전부 곱하면 됨(결과 -> 결과-1층 -> .... 현재층까지)
  def back_propagate(self,error):
    for i in reversed(range(len(self.derivatives))):#뒤쪽에서부터 돌아옴, 역전파는 레이어 사이에서 벌어지므로 레이어-1번 반복(=derivates 길이)
      activations = self.activations[i+1]#현재 위치의 활성화 함수, 뒤에서부터 시작됨
      delta = error * self._sigmoid_derivative(activations)#역 시그모이드 f(x)*(1-f(x))
      delta_reshaped = delta.reshape(delta.shape[0], -1).T #역 시그모이드를 통해 바뀐 행렬의 모양변경
      current_activations = self.activations[i]#해당 층의 가중치 배열을 가져옴
      current_activations_reshaped = current_activations.reshape(current_activations.shape[0],-1)#해당 가중치 배열을 반대로 변경
      print("현재 activation 크기는",current_activations_reshaped.shape)
      print("현재 델타 크기는",delta_reshaped.shape)

      self.derivatives[i] = np.dot(current_activations_reshaped, delta_reshaped)
      error=np.dot(delta,self.weights[i].T)
  #에러는 최종 결과값들 에러의 총합, 가중치 업데이트에는 합쳐지기 전 각 에러값들에 대한 결괏값이 필요함
  #에러의 총합 -> 합쳐지기 전 에러값 -> 적용된 시그모이드를 역함수로 다시 되돌림 -> 해당 에러가 현재 레이어에서 얼마나 영향을 받았나, 현재 에러의 지분


  def train(self,inputs,targets,epochs,learning_rate):
    for i in range(epochs):
      sum_error=0
      for j,input in enumerate(inputs):
        target=targets[j]
        output=self.forward_propagate(inputs)
        error=target-output
        self.back_propagate(error)
        self.gradient_descent(learning_rate)
        sum_error+=self.mse(target,output)
      print("Error: {} at epoch {}".format(sum_errors / len(items), i+1))
    print("트레이닝 완료")

  def gradient_descent(self,learning_rate=1):
    for i in range(len(self.weights)):#해당 레이어의 가중치 갯수(뉴런 갯수)만큼 반복
      weights=self.weights[i]
      derivatives=self.derivatives[i]
      weight+=derivatives*learning_rate

  def _sigmoid(self,x):#시그모이드 함수
    y=1.0/(1+np.exp(-x))
    return y

  def _sigmoid_derivative(self,x):#역 시그모이드 함수
    return x*(1.0-x)

  def _mse(self,target,output):#평균제곱오차
    return np.average((target-output)**2)


if __name__=="__main__":

  items = np.array([[random()/2 for _ in range(2)] for _ in range(1000)])
  targets = np.array([[i[0] + i[1]] for i in items])

  mlp.train(items, targets, 50, 0.1)

    # create dummy data
  input = np.array([0.3, 0.1])
  target = np.array([0.4])

    # get a prediction
  output = mlp.forward_propagate(input)

  print()
  print("Our network believes that {} + {} is equal to {}".format(input[0], input[1], output[0]))


Error: 0.00037670635870269687 at epoch 1
Error: 0.00037636591612477926 at epoch 2
Error: 0.0003760558169522757 at epoch 3
Error: 0.00037575164777837864 at epoch 4
Error: 0.00037545229507613364 at epoch 5
Error: 0.00037515700843509197 at epoch 6
Error: 0.0003748652534021549 at epoch 7
Error: 0.00037457662862947183 at epoch 8
Error: 0.0003742908214488524 at epoch 9
Error: 0.00037400758233048633 at epoch 10
Error: 0.0003737267089998846 at epoch 11
Error: 0.00037344803577949316 at epoch 12
Error: 0.0003731714259634921 at epoch 13
Error: 0.0003728967660960643 at epoch 14
Error: 0.0003726239615360078 at epoch 15
Error: 0.0003723529329457908 at epoch 16
Error: 0.0003720836134758501 at epoch 17
Error: 0.0003718159464882503 at epoch 18
Error: 0.0003715498837072351 at epoch 19
Error: 0.00037128538371199177 at epoch 20
Error: 0.0003710224107059064 at epoch 21
Error: 0.00037076093351035943 at epoch 22
Error: 0.0003705009247414326 at epoch 23
Error: 0.0003702423601359689 at epoch 24
Error: 0.000369